In [ ]:
import cv2
import requests
import numpy as np
from roboflow import Roboflow

# Initialize Roboflow API
rf = Roboflow(api_key="d1W6yFx7wbWgs7hjXQgz")
project = rf.workspace().project("platic-detector")
model = project.version("1").model

# Function to get predictions and draw bounding boxes
def process_frame(frame):
    # Resize frame to model's input size (usually 416x416 or similar)
    resized_frame = cv2.resize(frame, (640, 480))
    
    # Save the frame as a temporary image file
    temp_filename = "temp_image.jpg"
    cv2.imwrite(temp_filename, resized_frame)
    
    # Run the Roboflow model on the frame
    prediction = model.predict(temp_filename).json()
    
    # Loop over all predictions
    for prediction_obj in prediction['predictions']:
        # Get bounding box and class label
        x = int(prediction_obj['x'])
        y = int(prediction_obj['y'])
        width = int(prediction_obj['width'])
        height = int(prediction_obj['height'])
        class_name = prediction_obj['class']
        
        # Draw bounding box
        cv2.rectangle(frame, 
                      (x - width // 2, y - height // 2), 
                      (x + width // 2, y + height // 2), 
                      (0, 255, 0), 2)
        
        # Display class name and coordinates on frame
        label = f"{class_name} ({x}, {y})"
        cv2.putText(frame, label, (x - width // 2, y - height // 2 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    return frame

# Open webcam
cap = cv2.VideoCapture(1)

# Check if webcam is opened
if not cap.isOpened():
    print("Error: Could not open video source.")
    exit()

# Main loop to read frames and process them
while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Failed to capture frame.")
        break
    
    # Process frame for plastic detection
    processed_frame = process_frame(frame)
    
    # Display the resulting frame
    cv2.imshow('Plastic Detector', processed_frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import requests
import numpy as np
from roboflow import Roboflow

# Initialize Roboflow API
rf = Roboflow(api_key="d1W6yFx7wbWgs7hjXQgz")
project = rf.workspace().project("platic-detector")
model = project.version("1").model

# Function to calculate navigation instructions
def get_navigation_instructions(current_x, current_y, target_x, target_y):
    instructions = []
    
    if target_y > current_y:
        instructions.append("front")
    elif target_y < current_y:
        instructions.append("back")
    
    if target_x > current_x:
        instructions.append("right")
    elif target_x < current_x:
        instructions.append("left")
    
    return instructions

# Function to get predictions and draw bounding boxes
def process_frame(frame, point_coords):
    # Resize frame to model's input size (commonly used size 416x416)
    input_size = (416, 416)
    resized_frame = cv2.resize(frame, input_size)
    
    # Save the frame as a temporary image file
    temp_filename = "temp_image.jpg"
    cv2.imwrite(temp_filename, resized_frame)
    
    # Run the Roboflow model on the frame with a lower confidence threshold
    try:
        prediction = model.predict(temp_filename, confidence=30).json()  # Lowering confidence to 30%
    except Exception as e:
        print(f"Error during model prediction: {e}")
        return frame
    
    # Print the API response for debugging
   # print("Prediction Response:", prediction)

    # If no predictions were made, return the frame as is
    if not prediction['predictions']:
        # print("No predictions were made")
        return frame

    # Loop over all predictions
    for prediction_obj in prediction['predictions']:
        # Get bounding box and class label
        x = int(prediction_obj['x'])
        y = int(prediction_obj['y'])
        width = int(prediction_obj['width'] * 0.5)  # Make the bounding box smaller
        height = int(prediction_obj['height'] * 0.5)  # Make the bounding box smaller
        class_name = prediction_obj['class']
        
        # Draw smaller bounding box
        cv2.rectangle(frame, 
                      (x - width // 2, y - height // 2), 
                      (x + width // 2, y + height // 2), 
                      (0, 255, 0), 2)
        
        # Display class name and coordinates on frame
        label = f"{class_name} ({x}, {y})"
        cv2.putText(frame, label, (x - width // 2, y - height // 2 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        # Calculate and print navigation instructions
        instructions = get_navigation_instructions(point_coords[0], point_coords[1], x, y)
        print("Instructions:", instructions)

        # Update the point coordinates to move toward the target
        step_size = 5  # Speed of movement
        if point_coords[0] < x:
            point_coords[0] += min(step_size, abs(point_coords[0] - x))
        elif point_coords[0] > x:
            point_coords[0] -= min(step_size, abs(point_coords[0] - x))
        
        if point_coords[1] < y:
            point_coords[1] += min(step_size, abs(point_coords[1] - y))
        elif point_coords[1] > y:
            point_coords[1] -= min(step_size, abs(point_coords[1] - y))

    # Draw the moving point on the frame
    cv2.circle(frame, (point_coords[0], point_coords[1]), 5, (0, 0, 255), -1)

    return frame

# Open webcam
cap = cv2.VideoCapture(1)

# Check if webcam is opened
if not cap.isOpened():
    print("Error: Could not open video source.")
    exit()

# Initialize point starting at the center of the frame
point_coords = [320, 240]

# Main loop to read frames and process them
while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Failed to capture frame.")
        break
    
    # Process frame for plastic detection and navigation
    processed_frame = process_frame(frame, point_coords)
    
    # Display the resulting frame
    cv2.imshow('Plastic Detector', processed_frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


loading Roboflow workspace...
loading Roboflow project...
Instructions: ['back', 'left']
Instructions: ['back', 'left']
Instructions: ['front', 'left']
Instructions: ['back', 'left']
Instructions: ['back', 'left']
Instructions: ['left']
Instructions: ['left']
Instructions: ['left']
Instructions: ['front', 'left']
Instructions: ['front', 'left']
Instructions: ['back', 'left']
Instructions: ['front', 'left']
Instructions: ['back', 'left']
Instructions: ['front', 'left']
Instructions: ['back', 'left']
Instructions: ['back', 'left']
Instructions: ['back', 'left']
Instructions: ['front', 'left']
Instructions: ['front', 'left']
Instructions: ['front', 'left']
Instructions: ['front', 'left']
